In [1]:
# Import the required libraries and dependencies
import pandas as pd
import datetime as dt

In [2]:
# Create path to CSV file
csv_path = "Metro_Interstate_Traffic_Volume.csv"
# Import the CSV into a pandas DataFrame
df = pd.read_csv(csv_path)

In [3]:
# Convert 'date_time' column to datetime data type
# and reset index
df['date_time'] = pd.to_datetime(df['date_time'])
df.set_index('date_time', inplace=True)
df.sort_index(inplace=True)

In [ ]:
#The 'date_time' column entry is the top of the hour for each row.  Holidays
#are denoted in the 'holiday' column, but are only noted for the first hour of that day
#at 00:00:00 (midnight), and the rest of the rows say "None" for that day, despite taking
#place during the holiday

# Forward fill the holiday column to propagate the holiday information for all hours of the day
df['holiday'] = df.groupby(df.index.date)['holiday'].fillna(method='ffill')


In [5]:


# Extract year from the datetime index and add 'year' column
df['year'] = df.index.year

# Group the data by year and holiday, and calculate the average traffic volume
average_holiday_volume = df.groupby(['year', 'holiday'])['traffic_volume'].mean()

# Display the result
print(average_holiday_volume)

# Export the DataFrame to a CSV file
df.to_csv('average_holiday_volume.csv')


C:\Users\Matt\AppData\Local\Temp\ipykernel_10328\3017105560.py:2: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  df['holiday'] = df.groupby(df.index.date)['holiday'].fillna(method='ffill')
C:\Users\Matt\AppData\Local\Temp\ipykernel_10328\3017105560.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['holiday'] = df.groupby(df.index.date)['holiday'].fillna(method='ffill')
C:\Users\Matt\AppData\Local\Temp\ipykernel_10328\3017105560.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['holiday'] = d

year  holiday                  
2012  Christmas Day                1603.280000
      Columbus Day                 3422.291667
      Thanksgiving Day             1974.416667
      Veterans Day                 2963.440000
2013  Christmas Day                1527.625000
      Columbus Day                 3369.153846
      Independence Day             2098.166667
      Labor Day                    2438.045455
      Memorial Day                 1966.958333
      New Years Day                1659.041667
      State Fair                   3865.291667
      Thanksgiving Day             2097.173913
      Veterans Day                 3211.086957
      Washingtons Birthday         3117.041667
2014  Martin Luther King Jr Day    2793.130435
      Memorial Day                 2097.833333
      New Years Day                1637.454545
      Washingtons Birthday         2471.875000
2015  Christmas Day                1559.250000
      Columbus Day                 3747.035714
      Independence Day      

In [ ]:
df.head()